In [68]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

from millify import millify

In [69]:
from pathlib import Path
p = Path().cwd()
print(f"original File path: {p}")
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
print(f"Updated File Path to pick from Data Library: { stringpath}")
ngsimfile=stringpath + '/' + 'Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data.csv'
ngsim = pd.read_csv(ngsimfile,low_memory=False)
print(f"{ngsim.duplicated().sum()} duplicate values have been removed")
ngsim.drop_duplicates(inplace=True)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Updated File Path to pick from Data Library: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


In [ ]:
ngsim['v_length']= ngsim['v_length']*0.3048
ngsim['Space_Headway']= ngsim['Space_Headway']*0.3048
ngsim['v_Vel']= ngsim['v_Vel']*0.3048
ngsim['v_Acc']= ngsim['v_Acc']*0.3048
ngsim['Relative_Time']= ngsim['Global_Time'] - ngsim['Global_Time'].min() + 1
ngsim= ngsim.drop(columns=['Movement', 'Direction','Section_ID','Int_ID','D_Zone','O_Zone','Lane_ID','Following','v_Width','Total_Frames','Frame_ID'])


In [ ]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim['Preceding_Vehicle_Class'] = np.NaN
ngsim['Rear_to_Front_Space_Headway'] = np.NaN

ngsim["v_Class_Name"] = ngsim["v_Class"].map({1:"Motorcycle", 2: "Car", 3: "Heavy Vehicle"})

filtered_ngsim_U= ngsim[((ngsim['Location'] == 'us-101')) ]
#These two vehicles have a bad trajectory and keep on following each other in a single lane in a matter of 
filtered_ngsim_I= ngsim[(ngsim['Location'] == 'i-80') & ( ((ngsim['Vehicle_ID'] != 1748 )  & (ngsim['Preceding'] != 1734 )) &(  (ngsim['Vehicle_ID'] != 1734 )  & (ngsim['Preceding'] != 1748 )))]
#filtered_ngsim_I= ngsim[(ngsim['Location'] == 'i-80')]

In [ ]:
filtered_ngsim_U[((filtered_ngsim_U['Space_Headway'] == 0 ) & (filtered_ngsim_U['Preceding'] > 0 )) ]

,Vehicle_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,v_Class_Name
76361,99,1118847948500,12.942,1420.624,6452132.084,1872437.889,27.887139,1,187.007874,-0.688976,92,0.0,0.0,us-101,1117684929401,NaN,NaN,Motorcycle
107058,806,1118848229800,6.699,796.216,6451666.005,1872853.511,47.572178,2,30.085302,0.000000,814,0.0,0.0,us-101,1117685210701,NaN,NaN,Car
452347,366,1118848971900,12.613,835.132,6451695.487,1872819.732,22.965879,1,93.602362,19.553806,370,0.0,0.0,us-101,1117685952801,NaN,NaN,Motorcycle
3595333,2871,1118847820000,7.631,547.070,6451508.896,1872991.251,42.650919,2,31.692913,0.000000,2876,0.0,0.0,us-101,1117684800901,NaN,NaN,Car
5900431,2964,1118847852000,4.868,309.729,6451336.355,1873154.258,54.133858,2,28.182415,-36.745407,2956,0.0,0.0,us-101,1117684832901,NaN,NaN,Car


In [ ]:
filtered_ngsim_I[((filtered_ngsim_I['Space_Headway'] == 0 ) & (filtered_ngsim_I['Preceding'] > 0 )) ]

,Vehicle_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,v_Class_Name
5910808,2506,1113437493200,53.343,380.297,6042837.478,2133451.781,46.91601,2,37.828084,-0.853018,2512,0.0,0.00,i-80,1112274474101,NaN,NaN,Car
6759942,2491,1113437558500,17.161,598.884,6042774.601,2133664.163,50.19685,2,37.171916,0.393701,2486,0.0,0.00,i-80,1112274539401,NaN,NaN,Car
9365776,572,1113437894100,17.928,1485.487,6042644.407,2134541.664,46.91601,2,0.000000,0.000000,566,0.0,9999.99,i-80,1112274875001,NaN,NaN,Car
9803270,1394,1113437169900,76.566,444.910,6042852.523,2133518.460,48.55643,2,20.078740,0.000000,1407,0.0,0.00,i-80,1112274150801,NaN,NaN,Car


In [ ]:
filtered_vClass_U= filtered_ngsim_U[['Vehicle_ID','v_Class','v_length']].drop_duplicates().sort_values('v_Class')
v_Class_M_U     =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 1 )  ]['Vehicle_ID'])
v_Class_C_U     =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 2 )  ]['Vehicle_ID'])
v_Class_HV_U    =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 3 )  ]['Vehicle_ID'])


filtered_vClass_I= filtered_ngsim_I[['Vehicle_ID','v_Class','v_length']].drop_duplicates().sort_values('v_Class')
v_Class_M_I     =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 1 )  ]['Vehicle_ID'])
v_Class_C_I     =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 2 )  ]['Vehicle_ID'])
v_Class_HV_I    =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 3 )  ]['Vehicle_ID'])

In [ ]:
result = []

for i in filtered_ngsim_U.index:
    if filtered_ngsim_U['Preceding'][i] in v_Class_C_U:
        result.append('Car')
    elif filtered_ngsim_U['Preceding'][i] in v_Class_HV_U:
        result.append('Heavy Vehicle')
    elif filtered_ngsim_U['Preceding'][i] in v_Class_M_U:
        result.append('Motorcycle')
    else: 
        result.append(np.NaN)
filtered_ngsim_U['Preceding_Vehicle_Class'] = result

result = []

for i in filtered_ngsim_I.index:
    if filtered_ngsim_I['Preceding'][i] in v_Class_C_I:
        result.append('Car')
    elif filtered_ngsim_I['Preceding'][i] in v_Class_HV_I:
        result.append('Heavy Vehicle')
    elif filtered_ngsim_I['Preceding'][i] in v_Class_M_I:
        result.append('Motorcycle')
    else: 
        result.append(np.NaN)
filtered_ngsim_I['Preceding_Vehicle_Class'] = result

In [ ]:
space_headway_rear_to_front = []

for i in filtered_ngsim_I.index:
    space=filtered_ngsim_I['Space_Headway'][i]  - filtered_vClass_I['v_length'][filtered_vClass_I['Vehicle_ID']== filtered_ngsim_I['Preceding'][i]].mean()
    space_headway_rear_to_front.append(space)
    if i%500000==0:
        print(f"row count{millify(i, precision=2, drop_nulls=False)} processed")
filtered_ngsim_I['Rear_to_Front_Space_Headway'] = space_headway_rear_to_front

row count9.00M processed
row count9.50M processed
row count10.00M processed
row count10.50M processed
row count11.00M processed
row count11.50M processed


In [ ]:
space_headway_rear_to_front = []

for i in filtered_ngsim_U.index:
    space = filtered_ngsim_U['Space_Headway'][i]  - filtered_vClass_U['v_length'][filtered_vClass_U['Vehicle_ID']== filtered_ngsim_U['Preceding'][i]].mean()
    space_headway_rear_to_front.append(space)
    if i%500000==0:
        print(f"row count{millify(i, precision=2, drop_nulls=False)} processed")
        

filtered_ngsim_U['Rear_to_Front_Space_Headway'] = space_headway_rear_to_front

row count0.00 processed
row count500.00k processed
row count2.50M processed
row count3.50M processed
row count4.00M processed
row count5.00M processed
row count5.50M processed
row count6.00M processed
row count6.50M processed
row count7.50M processed
row count8.00M processed
row count8.50M processed


In [ ]:
filtered_ngsim_U['Vehicle_combination']= filtered_ngsim_U['Preceding_Vehicle_Class'] + '-' + filtered_ngsim_U['v_Class_Name']
filtered_ngsim_I['Vehicle_combination']= filtered_ngsim_I['Preceding_Vehicle_Class'] + '-' + filtered_ngsim_I['v_Class_Name']

In [ ]:
filtered_ngsim= pd.concat([filtered_ngsim_U , filtered_ngsim_I])

In [ ]:
print(filtered_ngsim_U.shape[0])
print(filtered_ngsim_I.shape[0])
print(filtered_ngsim.shape[0])

4098933
4555939
8654872


In [ ]:
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
ngsimfilteredfile=stringpath + '\\' + 'Cleaned_NGSIM_Data_Part_1.csv'
filtered_ngsim.to_csv(ngsimfilteredfile, index = False)

In [ ]:


#this is converted to feet data
filtered_vClass_U['v_length'][filtered_vClass_U['Vehicle_ID']== 500 ].mean()

16529    6.0960
10727    4.5720
31466    5.4864
Name: v_length, dtype: float64

In [ ]:
filtered_ngsim_U[:2]

,Vehicle_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,v_Class_Name,Vehicle_combination
0,515,1118848075000,30.034,188.062,6451203.729,1873252.549,42.650919,2,76.476378,6.725722,500,390.748031,5.11,us-101,1117685055901,Car,332.786527,Car,Car-Car
3,2127,1118847624800,19.632,1775.614,6452425.122,1872172.475,44.291339,2,123.097113,36.745407,2124,160.498688,1.30,us-101,1117684605701,Car,117.027559,Car,Car-Car
